# R Bootcamp Part 5

## stargazer, xtable, and fixed effects regressions


This bootcamp will help us get more comfortable using **stargazer** and **xtable** to produce high-quality results and summary statistics tables, and using `felm()` from the **lfe** package for regressions (both fixed effects and regular OLS).


For today, let's load a few packages and read in a dataset on residential water use for residents in Alameda and Contra Costa Counties. 

## Preamble
Here we'll load in our necessary packages and the data file

In [1]:
library(tidyverse)
library(haven)
library(lfe)
library(lmtest)
library(sandwich)
library(stargazer)
library(xtable)

# load in wateruse data, add in measure of gallons per day "gpd"
waterdata <- read_dta("wateruse.dta") %>%
    mutate(gpd = (unit*748)/num_days)
head(waterdata)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --
v ggplot2 3.2.1     v purrr   0.3.3
v tibble  2.1.3     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0
Warning message:
"package 'tidyr' was built under R version 3.6.2"Warning message:
"package 'dplyr' was built under R version 3.6.3"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'lfe' was built under R version 3.6.2"Loading required package: Matrix

Attaching package: 'Matrix'

The following objects are masked from 'package:tidyr':

    expand, pack, unpack

Loading required package: zoo
Warning message:
"package 'zoo' was built under R version 3.6.2"
Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Attaching package: 'lmtest'

The following object is masked from 'pac

hh,billingcycle,unit,num_days,zip,yearbuilt,homesize,num_baths,num_beds,num_rooms,homeval,lotsize,degree_days,precip,city,gpd
1,4,142,61,94506,1991,6055,5.5,5,15,4451858,1.0560147,759.7091,1.60,6,1741.246
1,6,83,62,94506,1991,6055,5.5,5,15,4451858,1.0560147,257.3720,105.86,6,1001.355
9,2,162,62,94563,1978,3400,3.0,4,9,2253082,1.8099862,810.0521,2.48,17,1954.452
9,5,150,61,94563,1978,3400,3.0,4,9,2253082,1.8099862,442.4290,51.93,17,1839.344
11,2,186,61,94507,2004,7453,4.5,3,15,3447285,0.9800046,795.5771,1.42,2,2280.787
11,5,94,58,94507,2004,7453,4.5,3,15,3447285,0.9800046,523.2530,44.66,2,1212.276


# xtable

`xtable` is a useful package for producing custom summary statistics tables. let's say we're interested in summarizing water use ($gpd$) and degree days ($degree\_days$) according to whether a lot is less than or greater than one acre ($lotsize_1$) or more than 4 acres ($lotsize_4$):

`homesize <- waterdata %>%
    select(hh, billingcycle, gpd, degree_days, lotsize) %>%
    drop_na() %>%
    mutate(lotsize_1 = ifelse((lotsize < 1), "< 1", ">= 1"),
           lotsize_4 = ifelse((lotsize > 4), "> 4", "<= 4"))
head(homesize)`


In [2]:
homesize <- waterdata %>%
    select(hh, billingcycle, gpd, degree_days, lotsize) %>%
    drop_na() %>%
    mutate(lotsize_1 = ifelse((lotsize < 1), "< 1", ">= 1"),
           lotsize_4 = ifelse((lotsize > 4), "> 4", "<= 4"))
head(homesize)

hh,billingcycle,gpd,degree_days,lotsize,lotsize_1,lotsize_4
1,4,1741.246,759.7091,1.0560147,>= 1,<= 4
1,6,1001.355,257.3720,1.0560147,>= 1,<= 4
9,2,1954.452,810.0521,1.8099862,>= 1,<= 4
9,5,1839.344,442.4290,1.8099862,>= 1,<= 4
11,2,2280.787,795.5771,0.9800046,< 1,<= 4
11,5,1212.276,523.2530,0.9800046,< 1,<= 4


We know how to create summary statistics for these two variables for both levels of $lotsize\_1$ and $lotsize\_4$ using `summarise()`:

`sumstat_1 <- homesize %>%
    group_by(lotsize_1) %>%
    summarise(mean_gpd = mean(gpd), 
              mean_degdays = mean(degree_days))
sumstat_1`

`sumstat_4 <- homesize %>%
    group_by(lotsize_4) %>%
    summarise(mean_gpd = mean(gpd), 
              mean_degdays = mean(degree_days))
sumstat_4`

In [3]:
sumstat_1 <- homesize %>%
    group_by(lotsize_1) %>%
    summarise(mean_gpd = mean(gpd), 
              mean_degdays = mean(degree_days))
sumstat_1

sumstat_4 <- homesize %>%
    group_by(lotsize_4) %>%
    summarise(mean_gpd = mean(gpd), 
              mean_degdays = mean(degree_days))
sumstat_4

lotsize_1,mean_gpd,mean_degdays
< 1,1490.199,568.8051
>= 1,1961.472,569.2840


lotsize_4,mean_gpd,mean_degdays
<= 4,1589.234,568.5734
> 4,2616.090,575.1284


And now we can use `xtable()` to put them into the same table!

`full <- xtable(cbind(t(sumstat_1), t(sumstat_4)))
rownames(full)[1] <- "Lotsize Group"
colnames(full) <- c("lotsize_1 = 1", "lotsize_1 = 0", "lotsize_4 = 0", "lotsize_4 =1")
full`

In [4]:
full <- xtable(cbind(t(sumstat_1), t(sumstat_4)))
rownames(full)[1] <- "Lotsize Group"
colnames(full) <- c("lotsize_1 = 1", "lotsize_1 = 0", "lotsize_4 = 0", "lotsize_4 =1")
full

,lotsize_1 = 1,lotsize_1 = 0,lotsize_4 = 0,lotsize_4 =1
Lotsize Group,< 1,>= 1,<= 4,> 4
mean_gpd,1490.199,1961.472,1589.234,2616.090
mean_degdays,568.8051,569.2840,568.5734,575.1284


We now have a table `full` that is an xtable object. 

We can also spit this table out in html or latex form if needed using the `print.xtable()` function on our xtable `full`. To get the html code for the table, specify `type = "html"`:

`print.xtable(full, type = "html")`

Copy and paste the html code here to see how it appears

# Stargazer

`stargazer` is a super useful package for producing professional-quality regression tables. While it defaults to producing LaTeX format tables (a typesetting language a lot of economists use), for use in our class we can also produce html code that can easily be copied into text cells and formatted perfectly.

## Stargazer for Summary Statistics Tables

Like `xtable`, we can use `stargazer` to produce attractive summary statistics tables. 

To produce the basic summary statistic table, we can just run `stargazer()` on our data frame in the following way:

`stargazer(as.data.frame(waterdata), type = "text")`

Note that we need to include the `as.data.frame()` around our dataset to make sure stargazer reads it in properly (if you forget this you'll get column names but no rows in the table).

By default, we get the number of observations, the mean, standard deviation, minimum, 25th and 27th percentiles, and max for each variable. 

You can change which statistics get displayed by including the `summary.stat` argument to specify the statistics we want, or using `omit.summary.stat` to hide the ones we don't want. For example, we can omit the 25th and 75th percentiles with the following two ways (and include the median in the first):

`stargazer(as.data.frame(waterdata), type = "text", summary.stat = c("n", "mean", "median", "sd", "min", "max"))`
`stargazer(as.data.frame(waterdata), type = "text", omit.summary.stat = c("p25", "p75"))`

You can also obtain the corresponding html code by specifying `type = "html"` and then copying + pasting the code into a markdownd cell:

`stargazer(as.data.frame(waterdata), type = "html", 
           summary.stat = c("n", "mean", "median", "sd", "min", "max"))`

Copy and paste the html code here!

## Stargazer for Regression Tables

We can also use `stargazer to produce high-quality regression tables. If we run the following three regressions: 

\begin{align*} GPD_{it} &= \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~(1)\\
 GPD_{it} &= \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} + \beta_3 lotsize_{i}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~(2)\\
GPD_{it} &= \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} + \beta_3 lotsize_{i} + \beta_4 Homeval_i~~~~~~~~~~~~~~~~~~(3)
\end{align*}

We might want to present the results side by side in the same table so that we can easily compare coefficients from one column to the other. To do that with `stargazer`, we can 

1. Run each regression, storing them in memory (here we'll use `felm()` to run OLS without fixed effects)
2. Run `stargazer(reg1, reg2, reg3, ..., type )` where the first arguments are all the regression objects we want in the table, and telling R what type of output we want

If we specify `type = "text"`, we'll get the table displayed directly in the output window:

`reg_a <- felm(gpd ~ degree_days + precip, waterdata)
 reg_b <- felm(gpd ~ degree_days + precip + lotsize, waterdata)
 reg_c <- felm(gpd ~ degree_days + precip + lotsize + homeval, waterdata)`

`stargazer(reg_a, reg_b, reg_c, type = "text")`

And if we specify `type = "html"`, we'll get html code that we need to copy and paste into a text/markdown cell:

`stargazer(reg_a, reg_b, reg_c, type = "html")`

Now all we need to do is copy and paste that html code from the output into a text cell and we've got our table!

(copy your code here)

And we get a nice looking regression table with all three models side by side! This makes it easy to see how the coefficient on lot size falls when we add in home value, letting us quickly figure out the sign of correlation between the two.

## Stargazer and Heteroskedasticity-Robust Standard Errors 

There are often times where you want to use heteroskedasticity-robust standard errors in place of the normal kind to account for situations where we might be worried about violating our homoskedasticity assumption. To add robust standard errors to our table, we'll need to access them from the felm object with `$rse` and add them to stargazer() with the `se` argument as a list:

`stargazer(reg_a, reg_b, reg_c, 
           type = "html",
          se = list(reg_a$rse, reg_b$rse, reg_c$rse),
          omit.stat = "f")`
          
Here we're adding the robust standard errors to `stargazer()` with the `se =` argument (combining them together in the right order as a list). I'm also omitting the overall F test at the bottom with `omit.stat = "f"` since we'd need to correct that too for heteroskedasticity. 

Try copying the code below to see how the standard errors change when we use robust standard errors:

Copy and paste the updated table code here.


Now that looks pretty good, though note that the less than signs in the note for signifcance labels don't appear right. This is because html is reading the < symbol as a piece of code and not the math symbol. To get around this, you can add dollar signs around the < signs in the html code for the note to have the signs display properly:

`<sup>*</sup>p $<$ 0.1; <sup>**</sup>p $<$ 0.05; <sup>***</sup>p $<$ 0.01</td>`

## Table Options

Stargazer has a ton of different options for customizing the look of our table with optional arguments, including
* `title` lets us add a custom title
* `column.labels` lets you add text labels to the columns
* `covariate.labels` lets us specify custom labels for all our variables other than the variable names. Specify each label in quotations in the form of a vector with `c()`
* `ci = TRUE` adds in confidence intervals (by default for the 10\% level, but you can change it to the 1\% level with `ci.level = 0.99`
* `intercept.bottom = FALSE` will move the constant to the top of the table
* `digits` lets you choose the number of decimal places to display
* `notes` lets you add some notes at the bottom
* `order` lets you change the order in which the independent variables appear in the table. This expects a numeric vector with the new ordering
* `dep.var.caption` lets you change the text of `Dependent variable:` at the top. If you set it to the empty character string `""` it will remove the caption entirely.

For example, we could customize the above table as

`stargazer(reg_a, reg_b, reg_c, type = "text",
          title = "Water Use, Weather, and Home Characteristics",
          column.labels = c("Weather", "With Lotsize", "With HomeVal"),
          order = c(1,5,4,3,2),         
          intercept.bottom = FALSE,
          dep.var.caption = "",
          digits = 2,
          note = "Isn't stargazer neat?"
          )`


# Fixed Effects Regression


Today we will practice with fixed effects regressions in __R__. We have two different ways to estimate the model, and we will see how to do both and the situations in which we might favor one versus the other.

Let's give this a try using the dataset `wateruse.dta`. The subset of households are high water users, people who used over 1,000 gallons per billing cycle. We have information on their water use, weather during the period, as well as information on the city and zipcode of where the home is located, and information on the size and value of the house.

Suppose we are interested in running the following panel regression of residential water use:

$$ GPD_{it} = \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} ~~~~~~~~~~~~~~~~~~~~~~~(1)$$

Where $GPD$ is the gallons used per day by household $i$ in billing cycle $t$, $degree\_days$ the count of degree days experienced by the household in that billing cycle (degree days are a measure of cumulative time spent above a certain temperature threshold), and $precip$ the amount of precipitation in millimeters.

`reg1 <- lm(gpd ~ degree_days + precip, data = waterdata)
summary(reg1)`

Here we obtain an estimate of $\hat\beta_1 = 0.777$, telling us that an additional degree day per billing cycle is associated with an additional $0.7769$ gallon used per day. These billing cycles are roughly two months long, so this suggests an increase of roughly 47 gallons per billing cycle. Our estimate is statistically significant at all conventional levels, suggesting residential water use does respond to increased exposure to high heat.

We estimate a statistically insignificant coefficient on additional precipitation, which tells us that on average household water use in our sample doesn't adjust to how much it rains.

We might think that characteristics of the home impact how much water is used there, so we add in some home controls:


$$ GPD_{it} = \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it}  + \beta_3 lotsize_{i} + \beta_4 homesize_i + \beta_5 num\_baths_i + \beta_6 num\_beds_i + \beta_7 homeval_i~~~~~~~~~~~~~~~~~~~~~~~(2)$$

`reg2 <- lm(gpd ~ degree_days + precip + lotsize + homesize + num_baths + num_beds + homeval, data = waterdata)
summary(reg2)`

Our coefficient on $degree\_days$ remains statistically significant and doesn't change much, so we find that $\hat\beta_1$ is robust to the addition of home characteristics. Of these characteristics, we obtain statistically significant coefficients on the size of the lot in acres ($lotsize$), the size of the home in square feet ($homesize$), and the number of bedrooms in the home ($num_beds$).

We get a curious result for $\hat\beta_6$: for each additional bedroom in the home we predict that water use will *fall* by 48 gallons per day. 

### Discussion: what might be driving this effect? 

Since there are likely a number of sources of omitted variable bias in the previous model, we think it might be worth including some fixed effects in our model. These will allow us to control for some of the unobserved sources of OVB without having to measure them directly!

## Method 1: Fixed Effects with lm() 

Up to this point we have been running our regressions using the `lm()` function. We can still use `lm()` for our fixed effects models, but it takes some more work and gets increasingly time-intensive as datasets get large.

Recall that we can write our general panel fixed effects model as 

$$ y_{it} = \beta x_{it} + \mathbf{a}_i + \mathbf{d}_t + u_{it} $$

* $y$ our outcome of interest, which varies in both the time and cross-sectional dimensions
* $x_{it}$ our set of time-varying unit characteristics
* $\mathbf{a}_i$ our set of unit fixed effects
* $\mathbf{d}_t$ our time fixed effects

We can estimate this model in `lm()` provided we have variables in our dataframe that correspond to each level of $a_i$ and $d_t$. This means we'll have to generate them before we can run any regression.

### Generating Dummy Variables

In order to include fixed effects for our regression, we can first generate the set of dummy variables that we want. For example, if we want to include a set of city fixed effects in our model, we need to generate them.

We can do this in a few ways.

1. First, we can use `mutate()` and add a separate line for each individual city:

`fe_1 <- waterdata %>%
    mutate(city_1 = as.numeric((city==1)),
           city_2 = as.numeric((city ==2)),
           city_3 = as.numeric((city ==3))) %>%
    select(hh, city, city_1, city_2, city_3)
head(fe_1)`

hh,city,city_1,city_2,city_3
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,6,0,0,0
1,6,0,0,0
9,17,0,0,0
9,17,0,0,0
11,2,0,1,0
11,2,0,1,0


This can be super tedious though when we have a bunch of different levels of our variable that we want to make fixed effects for. In this case, we have 27 different cities.

2. Alternatively, we can use the `spread()` function to help us out. Here we first group our data by household and billing cycle (our level of identification). Then we add in a constant variable `v` that is equal to one in all rows, and a copy of city that adds "city_" to the front of the city number. Then we pass the data to `spread`, telling it to split the variable `cty` into dummy variables for all its levels, with all the "false" cases filled with zeros.

`fe_2 <- waterdata %>%
select(city, billingcycle)`


`fe_2 <- group_by(fe_2, hh, billingcycle) %>%
 mutate(v = 1, cty = paste0("city_", city)) %>% 
   spread(cty, v, fill = 0)
head(fe_2)`

That is much easier! 

This is a useful approach if you want to produce summary statistics for the fixed effects (i.e. what share of the sample lives in each city), but isn't truly necessary.

Alternatively, we can tell R to read our fixed effects variables as factors:

`lm(gpd ~ degree_days + precip + factor(city), data = waterdata)`

`factor()` around $city$ tells R to split city into dummy variables for each unique value it takes. R will then drop the first level when we run the regression - in our case making the first city our omitted group.


`reg3 <- lm(gpd ~ degree_days + precip + factor(city), data = waterdata)
summary(reg3)`

Now we have everything we need to run the regression


$$ GPD_{it} = \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it}  + \mathbf{a}_i + \mathbf{d}_t~~~~~~~~~~~~~~~~~~~~~~~(2)$$

Where $\mathbf{a}_i$ are our city fixed effects, and $\mathbf{d}_t$ our billing cycle fixed effects:

`fe_reg1 <- lm(gpd ~ degree_days + precip + factor(city) + factor(billingcycle), data = waterdata)
summary(fe_reg1)`

__R__ automatically chose the first dummy variable for each set of fixed effect (city 1 and billing cycle 1) to leave out as our omitted group. 

Now that we account for which billing cycle we're in (i.e. whether we're in the winter or whether we're in the summer), we find that the coefficient on $degree\_days$ is now much smaller and statistically insignificant. This makes sense, as we were falsely attributing the extra water use that comes from seasonality to temperature on its own. Now that we control for the season we're in via billing cycle fixed effects, we find that deviations in temperature exposure during a billing cycle don't result in dramatically higher water use within the sample.

### Discussion: Why did we drop the home characteristics from our model?

## Method 2: Fixed Effects with felm()

Alternatively, we could do everything way faster using the `felm()` function from the package __lfe__. This package doesn't require us to produce all the dummy variables by hand. Further, it performs the background math way faster so will be much quicker to estimate models using large datasets and many variables.

The syntax we use is now 

`felm(y ~ x1 + x2 + ... + xk | FE_1 + FE_2 + ..., data = df)`

* The first section $y \sim x1 + x2 +... xk$ is our formula, written the same way as with `lm()` - but omitting the fixed effects
* We now add a `|` and in the second section we specify our fixed effects. Here we say $FE\_1 + FE\_2$ which tells __R__ to include fixed effects for each level of the variables $FE\_1$ and $FE\_2$.
* we add the data source after the comma, as before.

Let's go ahead and try this now with our water data model:

`fe_reg2 <- felm(gpd ~ degree_days + precip | city + billingcycle, data = waterdata)
summary(fe_reg2)`

And we estimate the exact same coefficients on $degree\_days$ and $precip$ as in the case where we specified everything by hand! We didn't have to mutate our data or add any variables. The one potential downside is that this approach doesn't report the fixed effects themselves by default. The tradeoff is that `felm` runs a lot faster than `lm`, especially with large datasets. 

We can also recover the fixed effects with getfe(): 

`getfe(fe_reg2, se = TRUE, robust = TRUE)`

the argument `se = TRUE` tells it to produce standard errors too, and `robust = TRUE` further indicates that we want heteroskedasticity-robust standard errors.


Note that this approach doesn't give you the same reference groups as before, but we get the same relative values. Note that before the coefficient on $city2$ was 301.7 and now is 73.9. But the coefficient on $city1$ is -227.8, and if we subtract $city1$ from $city2$ to get the difference in averages for city 2 relative to city 1 we get $73.9 - (-227.8) = 301.7$, the same as before!

# Fixed Effects Practice Question #1

#### From a random sample of agricultural yields Y (1000 dollars per acre) for region $i$ in year $t$ for the US, we have estimated the following eqation:

 \begin{align*} \widehat{\log(Y)}_{it} &=	0.49	+ .01 GE_{it} ~~~~ 	R^2 = .32\\
	&~~~~~(.11)	 ~~~~ (.01)                ~~~~  n = 1526       \end{align*}

#### (a) Interpret the results on the Genetically engineered ($GE$) technology on yields. (follow SSS= Sign Size Significance)

#### (b) Suppose $GE$ is used more on the West Coast, where crop yields are also higher. How would the estimated effect of GE change if we include a West Coast region dummy variable in the equation? Justify your answer.

#### (c) If we include region fixed effects, would they control for the factors in (b)? Justify your answer.

#### (d)  If yields have been generally improving over time and GE adoption was only recently introduced in the USA, what would happen to the coefficient of GE if we included year fixed effects?



# Fixed Effects Practice Question #2

#### A recent paper investigates whether advertisement for Viagra causes increases in birth rates in the USA.  Apparently, advertising for products, including Viagra, happens on TV and reaches households that have a TV within a Marketing region and does not happen in areas outside a designated marketing region. What the authors do is look at hospital birth rates in regions inside and near the advertising region border  and collect data on dollars per 100 people (Ads) for a certain time, and compare those to the birth rates in hospitals located outside and near the advertising region designated border. They conduct a panel data analysis and estimate the following model:

$$ Births_{it} = \beta_0 + \beta_1 Ads + \beta_2 Ads^2 + Z_i + M_t + u_{it}$$

#### Where $Z_i$ are zipcode fixed effects and $M_t$ monthly fixed effects.

#### (a) Why do the authors include Zip Code Fixed Effects? In particular, what would be a variable that they are controlling for when adding Zip Code fixed effects that could cause a problem when interpreting the marginal effect of ad spending on birth rates? What would that (solved) problem be?

#### (b) Why do they add month fixed effects? 

